In [ ]:
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import cartopy.crs as ccrs
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np
import cartopy.crs as ccrs
import matplotlib.patches as patches
import cmaps
import cartopy.feature as cfeature
from datetime import datetime,timedelta
from matplotlib.offsetbox import AnchoredText#关于修改text位置的
import cv2
from pygac import get_reader_class
import ast
from tqdm import tqdm
proj =ccrs.NorthPolarStereo(central_longitude=17.5)
from sklearn.neighbors import BallTree
from shapely.geometry import Point, Polygon,LineString
#经过实验，按经纬度插值的清晰度小于先转化为实际地理坐标系再插值的清晰度
import concurrent.futures
import pyproj
import matplotlib.style as mplstyle
from utiles import *

In [ ]:
def read_AVHRR(filepath,tle_name):
    tle_dir = r'Z:\NOAA_TLE'
    reader_cls = get_reader_class(filepath)
    reader = reader_cls(tle_dir=tle_dir, tle_name=tle_name)
    reader.read(filepath)
    channels = reader.get_calibrated_channels()
    lons, lats = reader.get_lonlat()
    #根据diffs来判断overlap的扫描线序号-----------------------------------------------------------
    diffs = reader.correct_scan_line_numbers()['diffs']
    overlap_mask = (diffs==0)
    IR = channels[overlap_mask]
    lons,lats = lons[overlap_mask,:],lats[overlap_mask,:]
    #异常扫描线去除-------------------------------------------------------------------------------
    masked_scanlines = np.isnan(IR[:,:,4]).sum(axis=1)>0
    IR = IR[~masked_scanlines]
    lons = lons[~masked_scanlines,:]
    lats = lats[~masked_scanlines,:]
    return IR,lons,lats

In [ ]:
def lonlat2xy(central_lon,central_lat,lons,lats):
    lons,lats = np.array(lons),np.array(lats)
    #x_distance = geodistance(central_lon,lats.flatten(),lons.flatten(),lats.flatten())，为什么不这么算
    #x1_distance = R * cos(lat1) * (lon1-central_lon)，x2_distance = R * cos(lat2) * (lon2-central_lon)
    #x1_distance-x2_distance = R * cos(lat1) * lon1+R * cos(lat2) * lon2+R*central_lon*(cos(lat1)-cos(lat2)),这使得坐标差随着central_lon的变化而变化
    #lons,lats需要是一维的
    x_distance = geodistance(central_lon,central_lat,lons.flatten(),central_lat)
    x_coords = np.where(lons.flatten()<central_lon,-x_distance,x_distance)
    y_distance = geodistance(central_lon,central_lat,central_lon,lats.flatten())
    y_coords = np.where(lats.flatten()<central_lat,-y_distance,y_distance)
    return x_coords,y_coords

In [ ]:
def minmax(IR,IR_min=210,IR_max=280):
    #归一化
    IR = np.where(IR<IR_min,IR_min,IR)
    IR = np.where(IR>IR_max,IR_max,IR)
    return (IR-IR_min)/(IR_max-IR_min)

In [ ]:
def deal_cloud(matched_info):
    #裁剪并重插值地表温度和AVHRR数据
    #读入AVHRR数据----------------------------------------------------------------------------
    IR,AVHRR_lons,AVHRR_lats = read_AVHRR(matched_info.filepath,matched_info.tle_name)
    central_lon,central_lat = matched_info.cloud_lon,matched_info.cloud_lat
#对地表温度和红外进行裁剪插值----------------------------------------------------------------------------------------------------------------------------
    grid_size,distance=800,2000
    #裁剪区域坐标
    points = np.concatenate([-distance*np.array(range(1,grid_size//2+1))[::-1],np.array([0]),distance*np.array(range(1,grid_size//2+1))])
    _, _,chosed_x,chosed_y = generate_rotated_grid(central_lon, central_lat,1,0,grid_size=grid_size,distance=distance)
    #ERA5数据经纬度转实际坐标
    polar_coords_xmin,polar_coords_ymin = [-802,-802]
    polar_coords_xmax,polar_coords_ymax = [802,802]
    AVHRR_x_coords,AVHRR_y_coords = lonlat2xy(central_lon,central_lat,AVHRR_lons.flatten(),AVHRR_lats.flatten())
    #减少插值范围
    AV_clip_bool  = (AVHRR_x_coords<polar_coords_xmax)& (AVHRR_x_coords>polar_coords_xmin)\
            &(AVHRR_y_coords<polar_coords_ymax)&(AVHRR_y_coords>polar_coords_ymin)
    AVHRR_coords = np.hstack([AVHRR_x_coords.reshape((-1,1)),AVHRR_y_coords.reshape((-1,1))])
    VCI_coords = np.hstack([chosed_x.reshape((-1,1)),chosed_y.reshape((-1,1))])
    trans_IR = griddata(AVHRR_coords[AV_clip_bool],IR[:,:,4].flatten()[AV_clip_bool],VCI_coords,method = 'linear').reshape((len(points),len(points)))
    nan_place = np.isnan(trans_IR)
    trans_IR255 = 255-np.uint8(minmax(trans_IR)*255)
    #根据距离远近去除模糊边缘,高于80°不要去除，会有撕裂痕
    if central_lat<80:
        tree = BallTree(AVHRR_coords, leaf_size=15)
        counts = tree.query_radius(VCI_coords, r=20, count_only=True).reshape((len(points),len(points)))
        trans_IR255[np.logical_or(nan_place,counts<1)] = 0
    trans_IR255[nan_place] = 0
    return trans_IR255[::-1]